# Обработка и генерация изображений
## Лекция №6. Метрики оценки моделей

### 1. Постановка задачи генерации

*****

<b>Дано:</b>
- Набор независимых одинаково распределенных случайных величин $\{X_i\}_{i=1}^{n} \in \mathcal{X}$  (напр, $\mathcal{X} \in \mathcal{R}^m$) из неизвестного распределения $\pi(x)$.\
- Выборка $\{x_i\}_{i=1}^{n}$, где $x_i$ - реализация случайной величины.

<b>Задача:</b> 
1. Оценить $\pi(x)$ по выборке
2. Генерировать новые элементы $x$ из $\pi(x)$

### 2. Генеративные модели

*****

<div>
    <img src=./imgs/generetive_models_zoo.drawio.png style=width:800px>
</div>

[Исаченко Р. Порождающие модели машинного обучения. МФТИ, 2023](https://www.youtube.com/playlist?list=PLk4h7dmY2eYHVCEMMMqdKes__ehs5mRtR)

In [54]:
!pip freeze > requirements.txt

In [1]:
# подключение библиотек
import os
from tqdm import tqdm

import numpy as np
import torch
from torch import nn
from PIL import Image 
import torchvision.transforms as transforms
from diffusers import KandinskyV22Pipeline, KandinskyV22PriorPipeline

from GPUtil import showUtilization as gpu_usage
import gc

In [ ]:
# контроль памяти GPU
gc.collect()
torch.cuda.empty_cache()
gpu_usage()

In [ ]:
# преобразование промптов в эмбеддинги

pipe_prior = KandinskyV22PriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-prior").to("cuda")

prompts = ["red cat",
           "surfer catches a wave",
           "the police detain the criminal",
           "Trump won the election",
           "a ufo has arrived on earth",
           "LA seaside party",
           "molten wall clock"]

image_embs, negative_image_embs = pipe_prior(prompts).to_tuple()

del pipe_prior

In [ ]:
# генерация изображений по эмбеддингам промптов

pipe = KandinskyV22Pipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder").to("cuda")

for index, (image_emb, negative_image_emb) in enumerate(zip(image_embs, negative_image_embs)):
    image = pipe(
        image_embeds=image_emb.reshape(1, -1),
        negative_image_embeds=negative_image_emb.reshape(1, -1),
        height=512,
        width=512,
        num_inference_steps=50,
    ).images
    image[0].save(f"{index + 1}.png")

In [2]:
# чтение сгенерированных и реальных изображений и преобразование в тензоры

path_gen = './imgs/generated'
path_orig = './imgs/original'

gen_file_paths = [path_gen + '/' + file_name for file_name in os.listdir(path_gen)]
orig_file_paths = [path_orig + '/' + file_name for file_name in os.listdir(path_orig)]

gen_tensors = torch.zeros((7, 3, 512, 512))
orig_tensors = torch.zeros((7, 3, 512, 512))

pil_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.PILToTensor()
    ])

for index, (orig_file_path, gen_file_path) in enumerate(zip(orig_file_paths, gen_file_paths)):
    gen_im = Image.open(gen_file_path)
    orig_im = Image.open(orig_file_path)
    gen_tensor = pil_transform(gen_im) / 256
    orig_tensor = pil_transform(orig_im) / 256

    gen_tensors[index] = gen_tensor
    orig_tensors[index] = orig_tensor

### 3. Классификация метрик

*****

Метрики оценки генеративных моделей:
+ Основанных на правдоподобии (likelihood-based):
    - Правдоподобие
+ Неявной плотности (likelihood-free):
    - IS (Inception score)
    - FID (Frechet Inception Distance)
    - Precision-Recall

Метрики оценки качества изображений:
- Low-level
    - PixCorr (Pixelwise Correlation)
    - SSIM (Structural Similarity Index Measure)
- High-level
    - CLIP (Contrastive Language-Image Pre-Training)
    - SwAV (Swapping Assignments between multiple Views)

### 4. Метрики оценки генеративных моделей

*****

Представим, что у нас есть предобученный классификатор (оракул), который выдает на выходе распределение классов $p(y|x)$.\
Мы хотим, чтобы изображения генеративной модели удовлетворяли двум критериям:
<ol>
    <li>Sharpness
        <div><img src=./imgs/sharpness.png style=width:800px> </div>
        
Каждый класс должен быть точно идентифицирован.\
Условное распределениен $p(y|x)$ должно иметь низкую энтропию.

<li>Diversity
    <div><img src=./imgs/diversity.png style=width:800px> </div>

Хотим, чтобы семплы были разнообразные, то есть мы могли генерировать семплы разных классов с равной вероятностью. \
Маргинальное распределение $p(y) = \int p(y|x) p(x) dx$ должно иметь высокую энтропию.
</ol>

[Stefano Ermon. Deep Generative Models. Stanford](https://deepgenerativemodels.github.io/)

<div><img src=./imgs/sharpness_diversity.webp style=width:800px> </div>

[David Mack. A simple explanation of the Inception Score. Medium](https://medium.com/octavian-ai/a-simple-explanation-of-the-inception-score-372dff6a8c7a)

#### 4.1 Inception Score
- Sharpness:
$$low \ H(y|x) = - \sum_{y} \int_x p(y, x) log(p|x) dx$$
- Diversity:
$$high \ H(y) = - \sum_{y} p(y) log(p(y))$$
- Inception Score:
\begin{split} 
IS & = exp(H(y) - H(y|x)) \\
   & = exp \left( - \sum_{y} p(y) log(p(y)) + \sum_{y} \int_x p(y, x) log(p|x) dx\right) \\
   & = exp \left( \sum_{y} \int_x  p(y, x) log\frac{p(y|x)}{p(y)} dx\right) \\
   & = exp \left( E_{x} \sum_{y}  p(y | x) log\frac{p(y|x)}{p(y)} \right) \\
   & = exp \left(  E_{x} KL(p(y|x) || p(y)) \right)
\end{split}
- Недостатки:
   - Если генеративная модель генерирует изображения с метками классов, отличных от меток классов классификатора (оракул), то IS будет низким.
   - Если генеративная модель генерирует по одному изображению на класс, IS будел высоким (не измеряется внутриклассовое разнообразие).
   
[Salimans T. Improved Techniques for Training GANs, 2016](https://arxiv.org/abs/1606.03498)

In [3]:
# подсчет метрик
from ignite.metrics import InceptionScore
from ignite.handlers import Engine

def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

tensors = {'original': orig_tensors, 
           'generated': gen_tensors}

for key in tensors.keys():
    metric = InceptionScore()
    metric.attach(default_evaluator, "is")

    state = default_evaluator.run([tensors[key]])
    print(f'IS on {key} data:', round(state.metrics["is"], 3))

c:\Users\kshch\Projects\Обработка и генерация изображений\.venv\Lib\site-packages\ignite\handlers\checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


IS on original data: 1.317
IS on generated data: 1.531


In [4]:
from torchmetrics.image.inception import InceptionScore

inception = InceptionScore(normalize=True)
inception.update((tensors['original']).to('cpu'))
print('IS on original data:', inception.compute())

inception = InceptionScore(normalize=True)
inception.update((tensors['generated']).to('cpu'))
print('IS on generated data:', inception.compute())

c:\Users\kshch\Projects\Обработка и генерация изображений\.venv\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


IS on original data: (tensor(1.), tensor(0.))
IS on generated data: (tensor(1.), tensor(2.4333e-08))


#### 4.2 Frechet Inception Distance

<b>Определение:</b> \
Производящая функция моментов случайной величины $\xi(t)$: $$M_{\xi}(t) = E e^{\xi t}$$

<b>Теорема:</b> \
Если $\pi(x)$ и $p(x|\theta)$ имеют производящие функции моментов тогда:
$$\pi(x) = p(x|\theta) \Leftrightarrow E_{\pi} x^k = E_{p} x^k \ \ \ \forall k \geq 1$$


<b>Frechet Inception Distance</b>
$$ FID(\pi, p) = ||m_{\pi} - m_p||_2^2 + Tr \left( \sum_\pi + \sum_p - 2 \sqrt{\sum_\pi \sum_p} \right)$$

<div><img src=./imgs/FIDvsIS.png style=width:800px> </div>

Недостатки:
- Требуется большой датасет
- Долгий процесс подсчета метрики
- Оценка только 2х моментов

[Heusel M. GANs Trained by a Two Time-Scale Update Rule Converge to a Local Nash Equilibrium. 2017](https://arxiv.org/abs/1706.08500)

In [5]:
from torchmetrics.image.fid import FrechetInceptionDistance

metric = FrechetInceptionDistance(feature=64, normalize=True)
metric.update((tensors['original']).to('cpu'), real=True)
metric.update((tensors['generated']).to('cpu'), real=False)
fid = metric.compute().item()
print(fid)

11.676093101501465


#### 4.3 Precision-Recall

<b>Идеи:</b>
1. Sharpness: высокая точность изображений
2. Diversity: высокая вариативность изображений

<div><img src=./imgs/precision_recall.png style=width:800px> </div>

<b>Проблема:</b> как определять, попала ли точка в распределение или нет (определение границ распределения)\

<b>Решение:</b>\
$S_{\pi} = \{x_i\}_i^n \sim \pi(x)$ - real samples\
$S_{p} = \{x_i\}_i^n \sim p(x)$ - generated samples

Множества эмбеддингов: \
$G_\pi = \{g_i\}_{i=1}^n$\
$G_p = \{g_i\}_{i=1}^n$

Определим бинарную функцию:
$$
f(g, G)=
    \begin{cases}
        1 & \text{if } \exists g' \in G: ||g - g'||_2 \leq ||g' - NN_k(g', G)||_2\\
        0 & \text{otherwise}
    \end{cases}
$$
<div><img src=./imgs/distrib_approx.png style=width:800px> </div>


Precision-recall:

$PREC(G_\pi, G_p) = \frac{1}{n} \sum_{g \in G_p} f(g, G_\pi)$ 

$REC(G_\pi, G_p) = \frac{1}{n} \sum_{g \in G_\pi} f(g, G_p)$

<div><img src=./imgs/prec_rec_comparising.png style=width:800px> </div>

[Kynkäänniemi T. Improved Precision and Recall Metric for Assessing Generative Models. 2019](https://arxiv.org/abs/1904.06991)

<b>Truncation Trick</b>

1. BigGAN: truncated normal sampling\
    $z \sim N(0, 1)$\
    $x = G(z)$\
    $p(z|\psi) = N(z | 0, 1) / \int_{-\infty}^{\psi} N(z |0, 1) dz$

<div style=display: flex; justify-content: center;>
    <img src=./imgs/truncation_trick.jpg style=width:500px> 
    <img src=./imgs/truncation_trick_threshold.png style=width:700px> 
</div>

2. StyleGAN \
$z' = \hat{z} + \psi \cdot (z - \hat{z}), \ \ \hat{z} = E_z z$

<div style=display: flex; justify-content: center;>
    <img src=./imgs/truncation_trick2.jpg style=width:500px> 
</div>

In [1]:
# https://github.com/kynkaat/improved-precision-and-recall-metric?tab=readme-ov-file


### 5. Метрики оценки качества изображений

*****

#### 5.1 Pixelwise Correlation

$$r_{XY} = \frac{cov_{XY}}{\sigma_X \sigma_Y} = \frac{\sum(X - \overline{X})(Y - \overline{Y})}{\sqrt{\sum(X - \overline{X})^2 \sum(Y - \overline{Y})^2}}$$

In [23]:
preprocess = transforms.Compose([
    transforms.Resize(425, interpolation=transforms.InterpolationMode.BILINEAR),
])

# Flatten images while keeping the batch dimension
all_images_flattened = preprocess(orig_tensors).reshape(len(orig_tensors), -1).cpu()
all_brain_recons_flattened = preprocess(gen_tensors).view(len(gen_tensors), -1).cpu()

print(all_images_flattened.shape)
print(all_brain_recons_flattened.shape)

corrsum = 0
for i in tqdm(range(7)):
    corrsum += np.corrcoef(all_images_flattened[i], all_brain_recons_flattened[i])[0][1]
corrmean = corrsum / 7

pixcorr = corrmean
print(pixcorr)

torch.Size([7, 541875])
torch.Size([7, 541875])


100%|██████████| 7/7 [00:00<00:00, 167.36it/s]

0.20993910889691952


#### 5.2 Structural Similarity Index Measure

<b>Идея:</b> движение скользящего окна по изображению с подсчетом метрики. Каждое окно разбивается на три компоненты: яркость, контраст и структура.\
<b>Замечание:</b> три компоненты независимы

1. Оценка яркости:
$$\mu_x = \frac{1}{N} \sum_{i=1}^N x_i$$

2. Оценка контраста (несмещенная оценка):
$$\sigma_x = \left( \frac{1}{N - 1} \sum_{i=1}^N (x_i - \mu_x)^2 \right) ^\frac{1}{2}$$

3. Нормализация:
$$ \frac{x - \mu_x}{\sigma_x}$$

1. Введем функции:
- $l(x, y)$ - функция сравнения яркостей $\mu_x$ и $\mu_y$
- $c(x, y)$ - функция сравнения констанста $\sigma_x$ и $\sigma_y$
- $s(x, y)$ - функция сравнения структур сигналов

  Определим функцию:
  $$S(x, y) = f(l(x, y), c(x, y), s(x, y))$$
  которая должна удовлетворять следующим критериям:
  - Симметричность: $S(x, y) = S(y, x)$
  - Ограниченность: $S(x, y) <= 1$
  - Единственность максимума:\
  $S(x, y) = 1 \Leftrightarrow x = y$ (в дискретном случае, $x_i = y_i$, $i=1, 2, ..N$) 



2. Определим функции:
  - Функция сравнения яркостей:
  $$l(x, y) = \frac{2 \mu_x \mu_y + c_1}{\mu_x^2 + \mu_y^2 + c_1} \ \ \text{где} \ \ c_1 = (K_1 L)^2$$

  - Функция сравнения контраста:
  $$c(x, y) = \frac{2 \sigma_x \sigma_y + c_2}{\sigma_x^2 + \sigma_y^2 + c_2}  \ \ \text{где} \ \ c_2 = (K_2 L)^2$$

  - Функция сравнения структуры:
  $$s(x, y) = \frac{\sigma_{xy} + c_3}{\sigma_x \sigma_y + c_3}  \ \ \text{где} \ \ c_3 = (K_3 L)^2$$
  
  * $\sigma_{xy} = \frac{1}{N - 1}\sum_{i=1}^N(x_i - \mu_x)(y_i - \mu_y)$
  * $L$ - динамический диапазон значений пикселей (255 для 8-битных изображений в оттенках серого)
  * $K_{1, 2, 3} << 1$ - малая константа
  

1. Комбинируем и доопределяем:
  $$SSIM(x, y) = [l(x, y)]^\alpha \cdot [c(x, y)]^\beta \cdot [s(x, y)]^\gamma \ \  \text{где} \ \ \alpha > 0, \ \beta > 0, \ \gamma > 0$$
  
  Определим:
  - $\alpha = \beta = \gamma = 1$
  - $c_3 = c_2 / 2$

  $$SSIM(x, y) = \frac{(2 \mu_x \mu_y + c_1)(2 \sigma_{xy} + c_2)}{(\mu_x^2 + \mu_y^2 + c_1)(\sigma_x^2 + \sigma_y^2 + c_2)}$$

  <b>Замечание:</b> 
  - Функции $l(x, y)$, $c(x, y)$, $s(x, y)$, $SSIM(x, y)$ удовлетворяет критериям для $S(x,y)$
  - Измеряется от -1(нет сходства) до 1(полное сходство). 

2. Усреднение по всем окнам
  $$MSSIM(X, Y) = \frac{1}{M} \sum_{j=1}^M SSIM(x_j, y_j)$$
  где
  - X, Y - изображения
  - $x_j, y_j$ - изображения в $j$-ом локальном окне


<div><img src='./imgs/mse_vs_ssim.png' style=width:600px> </div>


[Zhou Wang. Image Quality Assessment: From Error Visibility to Structural Similarity. 2004](https://www.researchgate.net/publication/3327793_Image_Quality_Assessment_From_Error_Visibility_to_Structural_Similarity)

Дополнительно:
- [Universal Quality Image Index (UQI)](https://ieeexplore.ieee.org/document/995823)
- [Complex Wavelet SSIM (CW-SSIM)](https://ieeexplore.ieee.org/document/5109651)
- [Multi-scale Structural Similarity Index (MS-SSIM)](https://ieeexplore.ieee.org/abstract/document/1292216)

In [52]:
from torchmetrics.image import StructuralSimilarityIndexMeasure

ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
ssim(tensors['generated'], tensors['original'])

tensor(0.2971)

In [64]:
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim

preprocess = transforms.Compose([
    transforms.Resize(425, interpolation=transforms.InterpolationMode.BILINEAR), 
])

img_gray = rgb2gray(preprocess(tensors['original']).permute((0,2,3,1)).cpu())
recon_gray = rgb2gray(preprocess(tensors['generated']).permute((0,2,3,1)).cpu())

ssim_score=[]
for im,rec in tqdm(zip(img_gray,recon_gray), total=7):
    ssim_score.append(ssim(rec, im, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=1.0))

ssim = np.mean(ssim_score)
print(ssim)

100%|██████████| 7/7 [00:00<00:00, 128.61it/s]

0.3093117644560009


#### 5.3 Contrastive Language-Image Pre-Training

<div>
    <img src='./imgs/clip.png' style=width:1000px>
</div>

[Radford A. Learning Transferable Visual Models From Natural Language Supervision. 2021](https://arxiv.org/pdf/2103.00020) \
[CLIP. GitHub](https://github.com/OpenAI/CLIP)

#### 5.4 Swapping Assignments between multiple Views

Еще метрики сравнения изображений:
- Mean Squared Error (MSE) & Root Mean Squared Error (RMSE)
- Peak Signal-to-Noise Ratio (PSNR)
- Erreur Relative Globale Adimensionnelle de Synthèse (ERGAS)
- Spatial Correlation Coefficient (SCC)
- Relative Average Spectral Error (RASE)
- Spectral Angle Mapper (SAM)
- Visual Information Fidelity (VIF)